# Importing packages


In [49]:
# !pip install emoji
# !pip install flair
import emoji
import pandas as pd
import re
import io
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
import matplotlib.pyplot as plt
from sklearn import metrics
from flair.data import Sentence
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MaxAbsScaler
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
import nltk
import spacy
import pickle

# Reading file

In [50]:
df_t = pd.read_csv('FINAL.csv',encoding= 'unicode_escape')

# Text Preprocessing

In [51]:
# preprocessing class
class TextPreprocessor_new():
    def __init__(self):
        self.contractions = { 
        "ain't": "am not",
        "aren't": "are not",
        "can't": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he would",
        "he'd've": "he would have",
        "he'll": "he will",
        "he's": "he is",
        "how'd": "how did",
        "how'll": "how will",
        "how's": "how is",
        "i'd": "i would",
        "i'll": "i will",
        "i'm": "i am",
        "i've": "i have",
        "isn't": "is not",
        "it'd": "it would",
        "it'll": "it will",
        "it's": "it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "must've": "must have",
        "mustn't": "must not",
        "needn't": "need not",
        "oughtn't": "ought not",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "she'd": "she would",
        "she'll": "she will",
        "she's": "she is",
        "should've": "should have",
        "shouldn't": "should not",
        "that'd": "that would",
        "that's": "that is",
        "there'd": "there had",
        "there's": "there is",
        "they'd": "they would",
        "they'll": "they will",
        "they're": "they are",
        "they've": "they have",
        "wasn't": "was not",
        "we'd": "we would",
        "we'll": "we will",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what will",
        "what're": "what are",
        "what's": "what is",
        "what've": "what have",
        "where'd": "where did",
        "where's": "where is",
        "who'll": "who will",
        "who's": "who is",
        "won't": "will not",
        "wouldn't": "would not",
        "you'd": "you would",
        "you'll": "you will",
        "you're": "you are",
        "thx"   : "thanks",
        "ain’t": "am not",
        "aren’t": "are not",
        "can’t": "cannot",
        "can’t’ve": "cannot have",
        "could’ve": "could have",
        "couldn’t": "could not",
        "couldn’t’ve": "could not have",
        "didn’t": "did not",
        "doesn’t": "does not",
        "don’t": "do not",
        "hadn’t": "had not",
        "hadn’t’ve": "had not have",
        "hasn’t": "has not",
        "haven’t": "have not",
        "he’d": "he would",
        "he’d’ve": "he would have",
        "he’ll": "he will",
        "he’s": "he is",
        "how’d": "how did",
        "how’ll": "how will",
        "how’s": "how is",
        "i’d": "i would",
        "i’ll": "i will",
        "i’m": "i am",
        "i’ve": "i have",
        "isn’t": "is not",
        "it’d": "it would",
        "it’ll": "it will",
        "it’s": "it is",
        "let’s": "let us",
        "ma’am": "madam",
        "mayn’t": "may not",
        "might’ve": "might have",
        "mightn’t": "might not",
        "must’ve": "must have",
        "mustn’t": "must not",
        "needn’t": "need not",
        "oughtn’t": "ought not",
        "shan’t": "shall not",
        "sha’n’t": "shall not",
        "she’d": "she would",
        "she’ll": "she will",
        "she’s": "she is",
        "should’ve": "should have",
        "shouldn’t": "should not",
        "that’d": "that would",
        "that’s": "that is",
        "there’d": "there had",
        "there’s": "there is",
        "they’d": "they would",
        "they’ll": "they will",
        "they’re": "they are",
        "they’ve": "they have",
        "wasn’t": "was not",
        "we’d": "we would",
        "we’ll": "we will",
        "we’re": "we are",
        "we’ve": "we have",
        "weren’t": "were not",
        "what’ll": "what will",
        "what’re": "what are",
        "what’s": "what is",
        "what’ve": "what have",
        "where’d": "where did",
        "where’s": "where is",
        "who’ll": "who will",
        "who’s": "who is",
        "won’t": "will not",
        "wouldn’t": "would not",
        "you’d": "you would",
        "you’ll": "you will",
        "you’re": "you are"
        }

        self.emoticons_str = r"""
            (?:
                [:=;] # Eyes
                [oO\-]? # Nose (optional)
                [D\)\]\(\]/\\OpP] # Mouth
            )"""

        self.regex_str = [
            self.emoticons_str,
            r'<[^>]+>', # HTML tags
            r'(?:@[\w_]+)', # @-mentions
            r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
            r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs

            r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
        ]

        self.tokens_re = re.compile(r'('+'|'.join(self.regex_str)+')', re.VERBOSE | re.IGNORECASE)
        
        self.spacy_nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner']) 
        self.spacy_nlp.add_pipe(self.remove_punctuations_emoji_symbol, name="stopwords", last=True)
    
   
    
    def remove_punctuations_emoji_symbol(self,doc):

        # This will remove stopwords and punctuation.
        # Use token.text to return strings, which we'll need for Gensim.
        doc = " ".join([token.text.lower() for token in doc ])
        doc = emoji.get_emoji_regexp().sub(r'', doc)  # get emoji free text
        doc=" ".join([self.contractions[token] if token in self.contractions.keys() else token for token in doc.split()]) # handle short words

        doc = self.tokens_re.sub(r'', doc)   
        doc = re.sub(r'\s\s+','', doc)
        doc = re.sub(r'[ ]{2, }',' ',doc)

        doc = re.sub(r'http\S+','', doc)
        doc = re.sub(r'@',' ',doc)
        doc = re.sub(r'#',' ',doc)

        return doc

    def transform(self, X):  
        
        
        X_new = self.spacy_nlp(X)
        return  X_new

#

In [52]:
x = 'text'
TextPreprocessor_new_obj = TextPreprocessor_new()
df_t[x] = df_t[x].apply(lambda x:TextPreprocessor_new_obj.transform(x) )

In [53]:
df_t.shape

(416, 12)

# Train , Test and Dev split

In [54]:
train = df_t[0:320]
dev = df_t[320:350]
test = df_t[350:]

In [55]:
train.to_csv("./data/train.csv",index=False)
dev.to_csv("./data/dev.csv",index=False)
test.to_csv("./data/test.csv",index=False)

In [56]:
train.columns

Index(['created_at', 'text', 'source', 'is_quote', 'favourites_count',
       'retweet_count', 'followers_count', 'friends_count',
       'account_created_at', 'verified', 'is_rumor', 'rumor_veracity'],
      dtype='object')

# Rumar Checking - Binary classification

## Step1: Preparing corpus

In [57]:
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus

In [58]:
data_folder = '/content/data'
# column format indicating which columns hold the text and label(s)
column_name_map = {1: "text", 10: "label_topic"}

# load corpus containing training, test and dev data and if CSV has a header, you can skip it
corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=True,
                                         delimiter=',',    # tab-separated files
) 

2020-06-28 03:48:15,061 Reading data from /content/data
2020-06-28 03:48:15,066 Train: /content/data/train.csv
2020-06-28 03:48:15,067 Dev: /content/data/dev.csv
2020-06-28 03:48:15,068 Test: /content/data/test.csv


## Step 2: Training model

In [59]:
from torch.optim.adam import Adam

from flair.data import Corpus
from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

In [60]:
# 2. create the label dictionary
label_dict = corpus.make_label_dictionary()

2020-06-28 03:48:30,535 Computing label dictionary. Progress:


100%|██████████| 386/386 [00:00<00:00, 657.80it/s]

2020-06-28 03:48:31,459 [b'False', b'True']


In [61]:

# 3. initialize transformer document embeddings (many models are available)
document_embeddings = TransformerDocumentEmbeddings('distilbert-base-uncased', fine_tune=True)

# 4. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

# 5. initialize the text classifier trainer with Adam optimizer
trainer = ModelTrainer(classifier, corpus, optimizer=Adam)

# 6. start the training
trainer.train('resources/Binary_classification',
              learning_rate=3e-5, # use very small learning rate
              mini_batch_size=16,
              mini_batch_chunk_size=4, # optionally set this if transformer is too much for your machine
              max_epochs=5, # terminate after 5 epochs
              )

2020-06-28 03:48:47,013 ----------------------------------------------------------------------------------------------------
2020-06-28 03:48:47,015 Model: "TextClassifier(
  (document_embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in

{'dev_loss_history': [0.17895552515983582,
  0.1749493032693863,
  0.24284397065639496,
  0.2100742757320404,
  0.43733978271484375],
 'dev_score_history': [0.9666666666666667, 0.9, 0.9, 0.9333333333333333, 0.9],
 'test_score': 0.9393939393939394,
 'train_loss_history': [0.5169056363403797,
  0.2738830089569092,
  0.06330912783741952,
  0.005324843525886536,
  0.002329032123088837]}

## Step 3:  Prediction on test data set

In [71]:
predicted=[]
classifier = TextClassifier.load('resources/Binary_classification/final-model.pt')
for i,row in test.iterrows():
  sent = Sentence(row["text"])
  classifier.predict(sent)
  predicted.append(bool(str(sent.labels[0]).split("(")[0].replace(" ","")))

2020-06-28 03:54:05,127 loading file resources/Binary_classification/final-model.pt


## Step 4: Classification Report

In [73]:
print(metrics.classification_report(test['is_rumor'].values, predicted))

              precision    recall  f1-score   support

        True       1.00      1.00      1.00        66

    accuracy                           1.00        66
   macro avg       1.00      1.00      1.00        66
weighted avg       1.00      1.00      1.00        66



# Rumour Veracity - Multiple classification

## Step1: Preparing corpus

In [74]:
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus

In [82]:
data_folder = '/content/data'
# column format indicating which columns hold the text and label(s)
column_name_map = {1: "text", 11: "label_topic"}

# load corpus containing training, test and dev data and if CSV has a header, you can skip it
corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=True,
                                         delimiter=',',    # tab-separated files
) 

2020-06-28 03:58:22,547 Reading data from /content/data
2020-06-28 03:58:22,552 Train: /content/data/train.csv
2020-06-28 03:58:22,553 Dev: /content/data/dev.csv
2020-06-28 03:58:22,554 Test: /content/data/test.csv


## Step 2: Training model

In [83]:
from torch.optim.adam import Adam

from flair.data import Corpus
from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

In [84]:
# 2. create the label dictionary
label_dict = corpus.make_label_dictionary()

2020-06-28 03:58:31,786 Computing label dictionary. Progress:


100%|██████████| 386/386 [00:00<00:00, 637.10it/s]

2020-06-28 03:58:32,732 [b'not_a_rumor', b'Unverified', b'TRUE', b'FALSE']


In [85]:

# 3. initialize transformer document embeddings (many models are available)
document_embeddings = TransformerDocumentEmbeddings('distilbert-base-uncased', fine_tune=True)

# 4. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

# 5. initialize the text classifier trainer with Adam optimizer
trainer = ModelTrainer(classifier, corpus, optimizer=Adam)

# 6. start the training
trainer.train('resources/MultiClassification',
              learning_rate=3e-5, # use very small learning rate
              mini_batch_size=16,
              mini_batch_chunk_size=4, # optionally set this if transformer is too much for your machine
              max_epochs=5, # terminate after 5 epochs
              )

2020-06-28 03:58:40,160 ----------------------------------------------------------------------------------------------------
2020-06-28 03:58:40,161 Model: "TextClassifier(
  (document_embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in

{'dev_loss_history': [1.2338298559188843,
  1.0922685861587524,
  1.3076529502868652,
  1.426966905593872,
  1.601783037185669],
 'dev_score_history': [0.6833333333333333,
  0.7833333333333333,
  0.7333333333333333,
  0.7333333333333333,
  0.7333333333333333],
 'test_score': 0.7348484848484849,
 'train_loss_history': [0.9153732985258103,
  0.5301170244812965,
  0.3525198459625244,
  0.12219503074884415,
  0.13419269770383835]}

## Step 3:  Prediction on test data set



In [86]:
predicted=[]
classifier = TextClassifier.load('resources/MultiClassification/final-model.pt')
for i,row in test.iterrows():
  sent = Sentence(row["text"])
  classifier.predict(sent)
  predicted.append(str(sent.labels[0]).split("(")[0].replace(" ",""))

2020-06-28 03:59:44,306 loading file resources/MultiClassification/final-model.pt


## Step 4: Classification Report

In [88]:
print(metrics.classification_report(test['rumor_veracity'].values, predicted))

              precision    recall  f1-score   support

       FALSE       0.68      0.58      0.63        45
        TRUE       0.25      0.33      0.29        15
  Unverified       0.00      0.00      0.00         6
 not_a_rumor       0.00      0.00      0.00         0

    accuracy                           0.47        66
   macro avg       0.23      0.23      0.23        66
weighted avg       0.52      0.47      0.49        66



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
